In [50]:
import pandas as pd
import json
import string
from textblob import TextBlob
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 


import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
import unicodedata
import re
from cleanco import basename
from thefuzz import fuzz
from name_matching.name_matcher import NameMatcher

In [49]:
# !pip install cleanco
# !pip install thefuzz
# !pip install name_matching

In [15]:
all_files_za = pd.read_csv("Classification/all_files_ZAv1.csv")

In [16]:
all_sources = all_files_za["source"].tolist()

In [17]:
all_sources

['300 Entertainment',
 'G59 Records',
 '031Choppa',
 'Squareball Ent. Limited',
 'Def Jam Recordings',
 'Def Jam Recordings',
 'Def Jam Recordings',
 'Def Jam Recordings',
 'Def Jam Recordings',
 'Def Jam Recordings',
 'Def Jam Recordings',
 'Gamebread, LLC / Def Jam Recordings',
 'Gamebread, LLC / Def Jam Recordings',
 'Gamebread, LLC / Def Jam Recordings',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic Records',
 'Slaughter Gang, LLC/Epic/Boominati Worldwide/Republic',
 'Slaughter Gang, LLC/Epic

In [18]:
tokens_words = nltk.word_tokenize(all_sources[0])
print(tokens_words)

['300', 'Entertainment']


In [19]:
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [45]:
lemmed_sources = []
for aword in all_sources:
    tokens_words = nltk.word_tokenize(aword)
    the_sentence = ""
    for singleword in tokens_words:
        new_word = ps.stem(singleword)
        new_word = lemmatizer.lemmatize(new_word)
        new_word = unicodedata.normalize('NFKD', new_word).encode('ASCII', 'ignore').decode()
        new_word = re.sub(r'[^\w\s]','',new_word)
        new_word = basename(new_word)
        the_sentence = the_sentence + " " + new_word
        the_sentence = the_sentence.lstrip()
    lemmed_sources.append(the_sentence)

In [56]:
all_files_za["lemmed sources"] = lemmed_sources

In [58]:
all_files_za[["source", "lemmed sources"]]

,source,lemmed sources
0,300 Entertainment,300 entertain
1,G59 Records,g59 record
2,031Choppa,031choppa
3,Squareball Ent. Limited,squarebal limit
4,Def Jam Recordings,def jam record
...,...,...
47481,"Homemade Projects, LLC / TenThousand Projects",homemad project tenthousand project
47482,"Homemade Projects, LLC / TenThousand Projects",homemad project tenthousand project
47483,Virgin,virgin
47484,Virgin,virgin


In [61]:
all_files_za[all_files_za["lemmed sources"] == "300 entertain"][["source", "lemmed sources"]]

,source,lemmed sources
0,300 Entertainment,300 entertain
15301,300 Entertainment,300 entertain
15302,300 Entertainment,300 entertain
15303,300 Entertainment,300 entertain
15304,300 Entertainment,300 entertain
...,...,...
46428,300 Entertainment,300 entertain
46429,300 Entertainment,300 entertain
46430,300 Entertainment,300 entertain
46431,300 Entertainment,300 entertain


In [66]:
unique_lemmed = all_files_za["lemmed sources"].unique().tolist()
unique_source = all_files_za["source"].unique().tolist()

In [67]:
len(unique_lemmed), len(unique_source)

(725, 732)

In [64]:
all_files_za.to_csv("all_files_za_lem.csv", index=False)

In [68]:
matcher = NameMatcher(top_n=10,
    lowercase=True,
    punctuations=True,
    remove_ascii=True,
    legal_suffixes=False,
    common_words=False,
    verbose=True)

matcher.set_distance_metrics(['discounted_levenshtein','SSK', 'fuzzy_wuzzy_token_sort'])

In [71]:
matcher.load_and_process_master_data('lemmed sources', all_files_za)

In [72]:
matcher.match_names(to_be_matched=all_files_za, column_matching='lemmed sources')

preprocessing...

preprocessing complete 
 searching for matches...



100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:29<00:00,  2.98s/it]


possible matches found   
 fuzzy matching...



100%|███████████████████████████████████████████████████████████████████████████| 47486/47486 [02:35<00:00, 305.88it/s]

done


,original_name,match_name,score,match_index
0,300 entertain,300 entertain,100.0,28842.0
1,g59 record,g59 record,100.0,1.0
2,031choppa,031choppa,100.0,2.0
3,squarebal limit,squarebal limit,100.0,3.0
4,def jam record,def jam record,100.0,28055.0
...,...,...,...,...
47481,homemad project tenthousand project,homemad project tenthousand project,100.0,47474.0
47482,homemad project tenthousand project,homemad project tenthousand project,100.0,47474.0
47483,virgin,virgin,100.0,28642.0
47484,virgin,virgin,100.0,28642.0


#### removing all capital letters

#### replace non-ASCII characters

#### we remove punctuation, i.e. remove any character that is not a word or space character with nothing

#### common legal business suffixes, using a package called cleanco

#### remove the most common words using regular expressions.

#### Cosine Similarity

#### Fuzzy String Matching

#### Post Processing

#### NameMatcher

In [17]:
a = ['CocaCola','CocaCola 3 Oz','Twix','Twix Caramel','Foldgers 3 Oz','Foldgers 10 Oz','Haagen Dazs Caramel','Black Forest Ham','Black Label Whiskey']

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
cv=CountVectorizer()
vect=cv.fit_transform(all_sources)
km=KMeans(n_clusters=500)

In [18]:
vectorised_names = km.fit_predict(vect)

In [19]:
len(vectorised_names)

47486

In [20]:
all_files_za["source vectorised"] = vectorised_names

In [21]:
all_files_za["source vectorised"].unique().tolist()

[69,
 38,
 76,
 72,
 494,
 104,
 86,
 98,
 10,
 3,
 150,
 77,
 33,
 138,
 184,
 197,
 355,
 281,
 142,
 83,
 36,
 316,
 263,
 176,
 492,
 214,
 277,
 196,
 118,
 73,
 228,
 464,
 30,
 241,
 117,
 378,
 178,
 46,
 375,
 253,
 110,
 204,
 49,
 265,
 24,
 274,
 34,
 12,
 369,
 80,
 475,
 436,
 341,
 346,
 26,
 455,
 470,
 162,
 4,
 437,
 229,
 259,
 149,
 130,
 146,
 352,
 143,
 92,
 161,
 14,
 326,
 135,
 392,
 2,
 61,
 93,
 63,
 457,
 465,
 57,
 27,
 175,
 377,
 111,
 325,
 484,
 438,
 333,
 231,
 65,
 307,
 132,
 35,
 203,
 495,
 357,
 490,
 108,
 205,
 54,
 208,
 9,
 43,
 356,
 219,
 55,
 469,
 350,
 381,
 87,
 441,
 382,
 224,
 364,
 120,
 183,
 88,
 351,
 136,
 267,
 202,
 182,
 171,
 144,
 101,
 456,
 106,
 123,
 217,
 254,
 127,
 486,
 315,
 141,
 112,
 70,
 354,
 164,
 410,
 454,
 360,
 261,
 308,
 252,
 272,
 216,
 84,
 20,
 384,
 477,
 481,
 339,
 406,
 483,
 188,
 296,
 210,
 348,
 51,
 434,
 288,
 148,
 6,
 91,
 125,
 180,
 413,
 107,
 391,
 367,
 52,
 297,
 145,
 56,
 169,
 

In [22]:
all_files_za[["source", 'source vectorised']]

,source,source vectorised
0,300 Entertainment,69
1,G59 Records,38
2,031Choppa,76
3,Squareball Ent. Limited,76
4,Def Jam Recordings,72
...,...,...
47481,"Homemade Projects, LLC / TenThousand Projects",383
47482,"Homemade Projects, LLC / TenThousand Projects",383
47483,Virgin,73
47484,Virgin,73


In [26]:
all_files_za[all_files_za['source vectorised'] == 69][["source", 'source vectorised']]

,source,source vectorised
0,300 Entertainment,69
15301,300 Entertainment,69
15302,300 Entertainment,69
15303,300 Entertainment,69
15304,300 Entertainment,69
...,...,...
46429,300 Entertainment,69
46430,300 Entertainment,69
46431,300 Entertainment,69
46432,300 Entertainment,69


In [28]:
all_files_za.to_csv("source_vect.csv", index=False)